# Run Perdido

In [ ]:
import os
from perdido.geoparser import Geoparser
from tqdm import tqdm
from lxml import etree

import warnings
warnings.filterwarnings('ignore')

from utils_functions import run_perdido

In [ ]:
geoparser = Geoparser(version='Standard', sources=['wiki_gaz'])

## Corpus Traitement_Texte

In [ ]:
path ='/Users/lmoncla/Documents/Data/Corpus/Choucas'
output_path = os.path.join(path ,'Perdido')

In [ ]:
for doc in tqdm(sorted(os.listdir(path))):
    try:
        num = int(doc.split('_')[-1])
        #print(num)
        filename = 'FichierNettoye_' + str(num)
        if os.path.exists(os.path.join(output_path, filename, filename + '.xml')):
            continue

        with open(os.path.join(path, doc, filename), 'r') as f:
            text = f.read()

            doc = run_perdido(text, output_path, filename, geoparser)
            if doc is not None:
                try:
                    os.mkdir(os.path.join(output_path, filename))
                except OSError:
                    pass
                
                doc.to_xml(os.path.join(output_path, filename + '.xml'))

                '''
                iob = 'n\tform\tlemma\tpos\ttype\n'
                for token in doc:
                        iob += token.tsv_format() + '\n'

                with open(os.path.join(output_path, filename, filename + '.tsv'), 'w') as f:
                    f.write(iob)

                doc.to_html(os.path.join(output_path, filename, filename + '_ent.html'), 'ent')
                doc.to_html(os.path.join(output_path, filename, filename + '_span.html'), 'span')
                '''
    except ValueError:
        pass

## Corpus Visorando

In [ ]:
path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/txt'
output_path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/tmp'

In [ ]:
for doc in tqdm(sorted(os.listdir(path))):
    if doc.endswith('.txt'):

        filename = doc[:-4]
        if os.path.exists(os.path.join(output_path, filename + '.xml')):
            continue

        with open(os.path.join(path, doc), 'r') as f:
            text = f.read()
            try:
                doc = run_perdido(text, output_path, filename, geoparser)
                if doc is not None:
                    doc.to_xml(os.path.join(output_path, filename + '.xml'))
            except:
                print(filename)
